# Установка библиотек

In [35]:
import pandas as pd #для работы с таблицами

Для автоматического применения статистического критерия (теста) используется модуль `stats` библиотеки `scipy`.

Этот модуль содержит большое количество вероятностных распределений, корреляционные функции и статистические тесты и многое другое.

Также активно используется библиотека `statsmodels`

Для установки библиотеки `scipy` и `statsmodels` можно воспользоваться менеджером пакетов `pip`.

In [36]:
!pip install scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [37]:
!pip install statsmodels

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/



# Загрузка данных

Для скачивания данных используем команду `!wget`. 

Данные появятся в локальной директории.

Мы будем использовать информацию о диаметре пицц разных производителей.

In [38]:
!wget https://raw.githubusercontent.com/harika-bonthu/Hypothesis-test-examples/main/pizzas.csv

--2022-09-07 12:29:52--  https://raw.githubusercontent.com/harika-bonthu/Hypothesis-test-examples/main/pizzas.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 539 [text/plain]
Saving to: ‘pizzas.csv.1’

pizzas.csv.1        100%[===================>]     539  --.-KB/s    in 0s      

2022-09-07 12:29:52 (24.4 MB/s) - ‘pizzas.csv.1’ saved [539/539]



Загрузим данные в память ноутбука.

In [39]:
data = pd.read_csv('pizzas.csv')

In [40]:
data.rename(columns={'Making Unit 1': 'Пиццерия №1', 'Making Unit 2': 'Пиццерия №2'}, inplace=True)

In [41]:
data.head()

,Пиццерия №1,Пиццерия №2
0,6.8090,6.7703
1,6.4376,7.5093
2,6.9157,6.7300
3,7.3012,6.7878
4,7.4488,7.1522


# Проверка данных на нормальность


In [42]:
H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально (мы отвергаем H0)'

Установим уровень значимости

In [43]:
alpha = 0.05

## Тест Шапиро-Уилка

In [44]:
from scipy.stats import shapiro

In [45]:
_, p = shapiro(data)
print('p=%.3f' % p)

# Интерпретация 

if p > alpha:
	print(H0)
else:
	print(Ha)

p=0.204
Данные распределены нормально


## Тест Д’Агостино

In [46]:
from scipy.stats import normaltest

In [47]:
_, p = normaltest(data)
#normaltest возвращает двустороннюю вероятность для проверки гипотезы

print('p=%.3f' % p[0])

# Интерпретация 

if p[0] > alpha/2:
	print(H0)
else:
	print(Ha)

p=0.251
Данные распределены нормально


# Независимый T-тест 

In [48]:
from scipy.stats import ttest_ind

In [49]:
H0 = 'Нет значимой разницы между диаметрами пиццы в разных пиццериях.'
Ha = 'Есть значимая разница между диаметрами пиццы в разных пиццериях.'

Зависимая переменная (диаметр пиццы) является количественной. Группы происходят из разных совокупностей. Следовательно, мы используем независимый T-тест.

In [50]:
def t_test(df):
    print('\n' + "*** Результаты независимого T-теста ***")
    test_results = ttest_ind(data['Пиццерия №1'], data['Пиццерия №2'], equal_var=True)

    p = round(test_results[1],2)

    if p>alpha:
        print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
    else:
        print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}")

t_test(data)


*** Результаты независимого T-теста ***
0.47 > 0.05. Мы не можем отвергнуть нулевую гипотезу. Нет значимой разницы между диаметрами пиццы в разных пиццериях.


-----

# Корреляция Спирмана 

In [51]:
from numpy.random import rand
from scipy.stats import spearmanr

data1 = rand(1000) * 20
data2 = data1 + (rand(1000) * 10)

corr, p = spearmanr(data1, data2)

In [52]:
print(corr,p)

0.8921604281604283 0.0


In [53]:
if p>alpha:
    print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу об отсутсвии зависимости между переменными.")
else:
    print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу об отсутсвии зависимости между переменными.")

0.0 <= 0.05. Мы отвергаем нулевую гипотезу об отсутсвии зависимости между переменными.


# ANOVA тест

In [54]:
from scipy.stats import f_oneway

Однофакторный дисперсионный анализ ANOVA проверяет нулевую гипотезу о том, что две или более групп имеют одинаковое среднее значение генеральной совокупности.

В качестве данных возьмем информацию по размерам раковины мидий, выращенных в разных местах.

In [55]:
petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764,
           0.0689]
tvarminne = [0.0703, 0.1026, 0.0956, 0.0973, 0.1039, 0.1045]

In [56]:
_, p = f_oneway(petersburg, magadan, tvarminne)

In [57]:
H0 = 'Нет значимой разницы между средним размером раковины мидий в трех разных местах.'
Ha = 'Есть значимая разница между средним размером раковины мидий в трех разных местах.'

In [58]:
if p>alpha:
  print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
else:
  print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}")

0.007870900547143829 <= 0.05. Мы отвергаем нулевую гипотезу. Есть значимая разница между средним размером раковины мидий в трех разных местах.


_____

# Домашнее задание

Используя информацию по мидиям Петербурга и Магадана, которые представлены в виде двух массивов petersburg и magadan в разделе ANOVA тест, проверьте данные на нормальность и на наличие корреляции. Для этого объедините 2 массива в DataFrame. Обоснуйте выбор теста на корреляцию. Сделайте вывод по гипотезе.
 
 
Также ответьте на вопрос:
Какова разница в среднем размере мидии в зависимости от города-производителя. Обоснуйте выбор теста. Сделайте вывод по гипотезе.


In [59]:
import pandas as pd

petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764, 0.0689]

print(f'nums in petersburg: {len(petersburg)}, nums in magadan: {len(magadan)}')

mussels_df = pd.DataFrame({
    'petersburg': petersburg,
    'magadan': magadan[0:len(petersburg)]
})

mussels_df

nums in petersburg: 7, nums in magadan: 8


,petersburg,magadan
0,0.0974,0.1033
1,0.1352,0.0915
2,0.0817,0.0781
3,0.1016,0.0685
4,0.0968,0.0677
5,0.1064,0.0697
6,0.1050,0.0764


#### Проверка на нормальность (тест Шапиро-Уилка)

In [60]:
from scipy.stats import shapiro

# Установим уровень значимости
alfa = 0.05

H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально (отвергаем нулевую гипотезу)'

s, p = shapiro(mussels_df)
print(f'p = {round(p,2)}')

# Интерпретация 
if p > alfa:
  print(H0)
else:
  print(Ha)  

p = 0.23
Данные распределены нормально


#### Проверка на нормальнось (тест Д'Агостино)

In [61]:
petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764, 0.0689]

# Для тестирования понадобится минимум 8 значений. 
# Увеличим массив с данными о размере мидий в Петербурге до 8, добавив среднее значение.
import statistics
p_mean = round(statistics.mean(petersburg),4)
petersburg.append(p_mean)

mussels_df2 = pd.DataFrame({
    'petersburg': petersburg,
    'magadan': magadan
})

# Установим уровень значимости
alfa = 0.05

H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально (отвергаем нулевую гипотезу)'

from scipy.stats import normaltest

s, p = normaltest(mussels_df2)
print(f'p = {round(p[0],2)}')

# Интерпретация теста
if p[0] > alfa/2:
  print(H0)
else:
  print(Ha)

p = 0.04
Данные распределены нормально


/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:1542: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=8
  "anyway, n=%i" % int(n))


#### Тест на наличие корреляции

In [62]:
# Установим уровень значимости
alpha = 0.05

petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764]

# На основе проведенных тестов (Шапиро-Уилка и Д'Агостино) мы можем утверждать, что данные распределены нормально.
# Таким образом для расчета корреляции мы использум стандартную корреляцию Пирсона
from scipy.stats import pearsonr

H0 = 'Зависимость между переменными отсутствует. Не можем отвергнуть нулевую гипотезу.'
Ha = 'Есть зависимость между переменныи. Отвергаем нулевую гипотезу'

corr, p = pearsonr(petersburg, magadan)
print(f'corr= {round(corr,2)}, p = {round(p,2)}')

# Интерпретация
if p>alpha:
    print(f"{round(p,2)} > {alpha}. {H0}")
else:
    print(f"{round(p,2)} <= {alpha}. {Ha}")

corr= 0.26, p = 0.58
0.58 > 0.05. Зависимость между переменными отсутствует. Не можем отвергнуть нулевую гипотезу.


#### Независимый T-тест

In [63]:
# Мы выбрали независимы T-тест исходя из следующего:
# a) данные распределены нормально
# b) тип независимой переменной - количественная
# c) сравниваются значения одной и той же переменной из разных 2ух совокупностей
# d) объектов в выборке < 30.

from scipy.stats import ttest_ind

alpha = 0.05
H0 = 'Нет значимой разницы между размерами мидий в Петербурге и Магадане.'
Ha = 'Есть значимая разница между размерами мидий в Петербурге и Магадане.'

def t_test(df):
    print('\n' + "*** Результаты независимого T-теста ***")
    p = ttest_ind(mussels_df['petersburg'], mussels_df['magadan'], equal_var=True)

    p = round(p[1],2)

    if p > alpha:
        print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
    else:
        print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}")

t_test(mussels_df)


*** Результаты независимого T-теста ***
0.01 <= 0.05. Мы отвергаем нулевую гипотезу. Есть значимая разница между размерами мидий в Петербурге и Магадане.


# Бонус

Скачаем датасет, содержащий информацию по давлению 120 человек.



In [64]:
!wget https://raw.githubusercontent.com/yug95/MachineLearning/master/Hypothesis%20testing/blood_pressure.csv

--2022-09-07 12:29:53--  https://raw.githubusercontent.com/yug95/MachineLearning/master/Hypothesis%20testing/blood_pressure.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2851 (2.8K) [text/plain]
Saving to: ‘blood_pressure.csv’

blood_pressure.csv  100%[===================>]   2.78K  --.-KB/s    in 0s      

2022-09-07 12:29:53 (51.5 MB/s) - ‘blood_pressure.csv’ saved [2851/2851]



In [65]:
data = pd.read_csv('blood_pressure.csv')

In [66]:
print('Размер выборки: ', data.shape)

Размер выборки:  (120, 5)


In [67]:
data.head()

,patient,sex,agegrp,bp_before,bp_after
0,1,Male,30-45,143,153
1,2,Male,30-45,163,170
2,3,Male,30-45,153,168
3,4,Male,30-45,153,142
4,5,Male,30-45,146,141


## Z-тест

Проведем Z-тест для проверки двух независимых групп данных на равенство средних значений выборок.

In [68]:
from statsmodels.stats import weightstats

In [75]:
_ ,p = weightstats.ztest(data['bp_before'], x2=data['bp_after'], value=0,alternative='two-sided')
print(float(p))


if p < 0.05:
    print("Мы отвергаем нулевую гипотезу об отсутсвии различий между данными.")
else:
    print("Мы не можем отвергнуть нулевую гипотезу об отсутсвии различий между данными.")

0.002162306611369422
Мы отвергаем нулевую гипотезу об отсутсвии различий между данными.


## Z-тест на пропорцию

Z-критерий одной пропорции используется для сравнения наблюдаемой пропорции с теоретической.

В этом тесте используются следующие нулевые гипотезы:

$H_0: p = p_0$ (доля мужчин, предоставивших данные о своем давлении, равна гипотетической пропорции $p_0$)

Альтернативная гипотеза может быть двусторонней, левосторонней или правосторонней:

$H_1 (двусторонний): p ≠ p_0$ (доля мужчин не равна некоторому гипотетическому значению $p_0$)
$H_1 (левосторонний): p < p_0$ (доля мужчин меньше некоторого гипотетического значения $p_0$)
$H_1 (правосторонний): p > p_0$ (доля мужчин больше некоторого гипотетического значения $p_0$)

Предположим, что доля мужчин в нашем датаcете равна 40%.

$p_0$: гипотетическая доля мужчин = 0.40

$x$: число мужчин в выборке мужчин: `len(data[data.sex == 'Male'])`

$n$: размер выборки = `len(data)`

Покажем, как использовать функцию `proportions_ztest` для выполнения  **z-теста**:

In [70]:
p_0 = 0.4
n = len(data)
x = len(data[data.sex == 'Male'])

print(n, x)

120 60


In [71]:
from statsmodels.stats.proportion import proportions_ztest

#perform one proportion z-test
_, p = proportions_ztest(count=x, nobs=n, value=p_0)

In [72]:
if p < 0.05:
    print("Мы отвергаем нулевую гипотезу о том, что доля мужчин, предоставивших данные о своем давлении, равна 40%.")
else:
    print("Мы не можем отвергнуть нулевую гипотезу.")

Мы отвергаем нулевую гипотезу о том, что доля мужчин, предоставивших данные о своем давлении, равна 40%.
